In [1]:
import os
import numpy as np
from nanowire.optics.simulate import Simulator
from nanowire.optics.postprocess import Simulation
from nanowire.optics.utils.utils import setup_sim
from nanowire.optics.utils.config import Config
import scipy.constants as consts
import scipy.integrate as intg
import IPython.display as disp
import matplotlib.pyplot as plt
%load_ext autoreload 
%autoreload 2

# Utility Functions

In [2]:
def integrate(arr, layer_obj, sim_proc):
    arr_slice = arr[layer_obj.slice]
    zsamps = layer_obj.iend - layer_obj.istart
    z_vals = np.linspace(0, layer_obj.thickness, zsamps)
    x_vals = np.linspace(0, sim_proc.period, sim_proc.x_samples)
    y_vals = np.linspace(0, sim_proc.period, sim_proc.y_samples)
    z_integral = intg.trapz(arr_slice, x=z_vals, axis=0)
    x_integral = intg.trapz(z_integral, x=x_vals, axis=0)
    y_integral = intg.trapz(x_integral, x=y_vals, axis=0)
    return y_integral

def compute_fluxes(sim):
    fluxes = sim.flux_dict
    total_incident_power = .5*sim.period**2/Zo*np.absolute(fluxes['Air'][0])
    total_reflected_power = .5*sim.period**2/Zo*np.absolute(fluxes['Air'][1])
    total_transmitted_power = .5*sim.period**2/Zo*np.absolute(sum(fluxes['Substrate_bottom']))
    total_absorbed_power = total_incident_power - total_reflected_power - total_transmitted_power
    print('Total Incident Power = {}'.format(total_incident_power))
    print('Total Reflected Power = {}'.format(total_reflected_power))
    print('Total Transmitted Power = {}'.format(total_transmitted_power))
    print('Total Absorbed Power = {}'.format(total_absorbed_power))
    summed_absorbed_power = 0
    abs_dict_fluxmethod = {}
    for layer, (forw_top, back_top) in fluxes.items():
        if '_bottom' in layer:
            continue
        bottom = layer+'_bottom'
        forw_bot, back_bot = fluxes[bottom] 
        print('-'*25)
        print('Layer: {}'.format(layer))
        print('Forward Top: {}'.format(forw_top))
        print('Backward Top: {}'.format(back_top))
        print('Forward Bottom: {}'.format(forw_bot))
        print('Backward Bottom: {}'.format(back_bot))
        P_in = forw_top + -1*back_bot
        P_out = forw_bot + -1*back_top
        print('Power Entering Layer: {}'.format(P_in))
        print('Power Leaving Layer: {}'.format(P_out))
        P_lost = P_in - P_out
        P_abs = .5*P_lost*(sim.period**2)/Zo
        abs_dict_fluxmethod[layer] = P_abs 
        print('Absorbed in Layer: {}'.format(P_abs))
        summed_absorbed_power += P_abs
    print('-'*25)
    print('Summed Absorption= {}'.format(summed_absorbed_power))
    return abs_dict_fluxmethod

# NW w/ Shell

In [3]:
conf = Config('AbsorptionTest.yml')
sim = Simulator(conf)
sim = setup_sim(sim)
Zo = consts.physical_constants['characteristic impedance of vacuum'][0]

In [4]:
sim.get_fluxes()

{'Air': ((8+0j), (-0.6889811691985808+0j)),
 'Air_bottom': ((8+0j), (-0.6889811691985808+0j)),
 'ITO': ((7.418002620063473-1.8038220179816822e-05j),
  (-0.10698378926173845+1.8038220179816822e-05j)),
 'ITO_bottom': ((6.942707091245627-0.004415115410193238j),
  (-0.10822149007644298+0.004415115410193238j)),
 'NW_AlShell': ((6.859400444864055+0.0013275440659821502j),
  (-0.024914843694923305-0.0013275440659821502j)),
 'NW_AlShell_bottom': ((2.466916596962712+0.004273111972457111j),
  (-0.08333950032467816-0.004273111972457111j)),
 'Substrate': ((2.3835770966380476+0j), 0j),
 'Substrate_bottom': ((0.3544751964486742+0j), 0j)}

In [5]:
abs_dict_fluxmethod = compute_fluxes(sim)

Total Incident Power = 0.0006636046823595181
Total Reflected Power = 5.71513912422142e-05
Total Transmitted Power = 2.940392501795628e-05
Total Absorbed Power = 0.0005770493660993477
-------------------------
Layer: Substrate
Forward Top: (2.3835770966380476+0j)
Backward Top: 0j
Forward Bottom: (0.3544751964486742+0j)
Backward Bottom: 0j
Power Entering Layer: (2.3835770966380476+0j)
Power Leaving Layer: (0.3544751964486742+0j)
Absorbed in Layer: (0.00016831519024378298+0j)
-------------------------
Layer: ITO
Forward Top: (7.418002620063473-1.8038220179816822e-05j)
Backward Top: (-0.10698378926173845+1.8038220179816822e-05j)
Forward Bottom: (6.942707091245627-0.004415115410193238j)
Backward Bottom: (-0.10822149007644298+0.004415115410193238j)
Power Entering Layer: (7.526224110139916-0.0044331536303730545j)
Power Leaving Layer: (7.049690880507365-0.0044331536303730545j)
Absorbed in Layer: (3.952871031050804e-05+0j)
-------------------------
Layer: NW_AlShell
Forward Top: (6.859400444864

In [6]:
sim.get_field()

In [7]:
sim_proc = Simulation(simulator=sim)
freq = sim_proc.conf[('Simulation', 'params', 'frequency','value')]
try:
    Esq = sim_proc.data['normEsquared']
except KeyError:
    Esq = sim_proc.normEsquared()
abs_dict_intmethod = {}
for layer_name, layer_obj in sim_proc.layers.items():
    print("Layer: {}".format(layer_name))
    base_unit = sim_proc.conf[('Simulation', 'base_unit')]
    n_mat, k_mat = layer_obj.get_nk_matrix(freq)
    # n and k could be functions of space, so we need to multiply the
    # fields by n and k before integrating
    res = integrate(Esq*n_mat*k_mat,layer_obj, sim_proc)
    p_abs_imag = 2*np.pi*freq*consts.epsilon_0*res*base_unit
    abs_dict_intmethod[layer_name] = p_abs_imag
    disp.display_latex("$P_{abs} = \\frac{\omega}{2} Im(\epsilon) \int |E|^2 dV"+" = {}$".format(p_abs_imag), raw=True)

Layer: Air


Layer: ITO


Layer: NW_AlShell


Layer: Substrate


In [8]:
print(abs_dict_fluxmethod)
print(abs_dict_intmethod)

{'Substrate': (0.00016831519024378298+0j), 'ITO': (3.952871031050804e-05+0j), 'NW_AlShell': (0.0003692054655450796+0j), 'Air': 0j}
{'Substrate': 0.0001682528522306985, 'ITO': 3.9477618786152697e-05, 'NW_AlShell': 0.00038622864783302726, 'Air': 0.0}


In [9]:
for key in abs_dict_fluxmethod.keys():
    fm = abs_dict_fluxmethod[key]
    im = abs_dict_intmethod[key]
    diff = fm - im
    try:
        pdiff = 100*abs(diff)/fm
    except ZeroDivisionError:
        pdiff = None
        pass
    print('-'*25)
    print("Layer: {}".format(key))
    print("Flux Method: {}".format(fm))
    print("Integral Method: {}".format(im))
    print("Diff: {}".format(diff))
    print("Percent Diff: {}".format(pdiff))

-------------------------
Layer: Substrate
Flux Method: (0.00016831519024378298+0j)
Integral Method: 0.0001682528522306985
Diff: (6.233801308448217e-08+0j)
Percent Diff: (0.03703647483877929+0j)
-------------------------
Layer: ITO
Flux Method: (3.952871031050804e-05+0j)
Integral Method: 3.94776187861527e-05
Diff: (5.109152435534513e-08+0j)
Percent Diff: (0.12925168555717667+0j)
-------------------------
Layer: NW_AlShell
Flux Method: (0.0003692054655450796+0j)
Integral Method: 0.00038622864783302726
Diff: (-1.7023182287947665e-05+0j)
Percent Diff: (4.610761182209301+0j)
-------------------------
Layer: Air
Flux Method: 0j
Integral Method: 0.0
Diff: 0j
Percent Diff: None


# NW w/ Shell w/ Lanczos

In [10]:
conf = Config('AbsorptionTest.yml')
conf[('Solver', 'LanczosSmoothing')] = True
sim = Simulator(conf)
sim = setup_sim(sim)
Zo = consts.physical_constants['characteristic impedance of vacuum'][0]

In [11]:
sim.get_fluxes()

{'Air': ((8+0j), (-0.763456782565338+0j)),
 'Air_bottom': ((8-1.1102230246251565e-16j),
  (-0.763456782565338+1.1102230246251565e-16j)),
 'ITO': ((7.458803815629436+0.0006962511562829121j),
  (-0.22226059819445257-0.0006962511562829121j)),
 'ITO_bottom': ((6.985538323185078-0.006973473057033053j),
  (-0.22701755175190058+0.006973473057033053j)),
 'NW_AlShell': ((6.810464916302812-0.0054596768019885555j),
  (-0.051944144869725394+0.0054596768019885555j)),
 'NW_AlShell_bottom': ((2.655213701219679-0.012739572445996467j),
  (-0.15086594283932253+0.012739572445996467j)),
 'NW_SiO2': ((2.7392398385383196-0.01337212743112065j),
  (-0.2348920801579685+0.01337212743112065j)),
 'NW_SiO2_bottom': ((2.7071301062475586-0.00587427638012096j),
  (-0.23743488369627014+0.00587427638012096j)),
 'Substrate': ((2.4696952225512665+0j), 0j),
 'Substrate_bottom': ((0.3839830328489277+0j), 0j)}

In [12]:
abs_dict_fluxmethod = compute_fluxes(sim)

Total Incident Power = 0.0006636046823595181
Total Reflected Power = 6.332918696118635e-05
Total Transmitted Power = 3.185161731814463e-05
Total Absorbed Power = 0.0005684238780801871
-------------------------
Layer: NW_AlShell
Forward Top: (6.810464916302812-0.0054596768019885555j)
Backward Top: (-0.051944144869725394+0.0054596768019885555j)
Forward Bottom: (2.655213701219679-0.012739572445996467j)
Backward Bottom: (-0.15086594283932253+0.012739572445996467j)
Power Entering Layer: (6.961330859142135-0.018199249247985022j)
Power Leaving Layer: (2.7071578460894044-0.018199249247985022j)
Absorbed in Layer: (0.0003528861413786614+0j)
-------------------------
Layer: NW_SiO2
Forward Top: (2.7392398385383196-0.01337212743112065j)
Backward Top: (-0.2348920801579685+0.01337212743112065j)
Forward Bottom: (2.7071301062475586-0.00587427638012096j)
Backward Bottom: (-0.23743488369627014+0.00587427638012096j)
Power Entering Layer: (2.97667472223459-0.01924640381124161j)
Power Leaving Layer: (2.942

In [13]:
sim.get_field()

In [14]:
sim_proc = Simulation(simulator=sim)
freq = sim_proc.conf[('Simulation', 'params', 'frequency','value')]
try:
    Esq = sim_proc.data['normEsquared']
except KeyError:
    Esq = sim_proc.normEsquared()
abs_dict_intmethod = {}
for layer_name, layer_obj in sim_proc.layers.items():
    print("Layer: {}".format(layer_name))
    base_unit = sim_proc.conf[('Simulation', 'base_unit')]
    n_mat, k_mat = layer_obj.get_nk_matrix(freq)
    # n and k could be functions of space, so we need to multiply the
    # fields by n and k before integrating
    res = integrate(Esq*n_mat*k_mat,layer_obj, sim_proc)
    p_abs_imag = 2*np.pi*freq*consts.epsilon_0*res*base_unit
    abs_dict_intmethod[layer_name] = p_abs_imag
    disp.display_latex("$P_{abs} = \\frac{\omega}{2} Im(\epsilon) \int |E|^2 dV"+" = {}$".format(p_abs_imag), raw=True)

Layer: Air


Layer: ITO


Layer: NW_AlShell


Layer: NW_SiO2


Layer: Substrate


In [15]:
print(abs_dict_fluxmethod)
print(abs_dict_intmethod)

{'ITO': (3.96522416799685e-05+0j), 'Air': 0j, 'NW_AlShell': (0.0003528861413786614+0j), 'Substrate': (0.00017301104689259944+0j), 'NW_SiO2': (2.87444812897463e-06+0j)}
{'ITO': 3.97260934053305e-05, 'NW_AlShell': 0.0003771534225778344, 'NW_SiO2': 3.6733446094859288e-06, 'Air': 0.0, 'Substrate': 0.0001727906720392508}


In [16]:
for key in abs_dict_fluxmethod.keys():
    fm = abs_dict_fluxmethod[key]
    im = abs_dict_intmethod[key]
    diff = fm - im
    try:
        pdiff = 100*abs(diff)/fm
    except ZeroDivisionError:
        pdiff = None
        pass
    print('-'*25)
    print("Layer: {}".format(key))
    print("Flux Method: {}".format(fm))
    print("Integral Method: {}".format(im))
    print("Diff: {}".format(diff))
    print("Percent Diff: {}".format(pdiff))

-------------------------
Layer: ITO
Flux Method: (3.96522416799685e-05+0j)
Integral Method: 3.97260934053305e-05
Diff: (-7.385172536200277e-08+0j)
Percent Diff: (0.18624855048059277+0j)
-------------------------
Layer: Air
Flux Method: 0j
Integral Method: 0.0
Diff: 0j
Percent Diff: None
-------------------------
Layer: NW_AlShell
Flux Method: (0.0003528861413786614+0j)
Integral Method: 0.0003771534225778344
Diff: (-2.4267281199173013e-05+0j)
Percent Diff: (6.876801991816737+0j)
-------------------------
Layer: Substrate
Flux Method: (0.00017301104689259944+0j)
Integral Method: 0.0001727906720392508
Diff: (2.203748533486326e-07+0j)
Percent Diff: (0.12737617470486456+0j)
-------------------------
Layer: NW_SiO2
Flux Method: (2.87444812897463e-06+0j)
Integral Method: 3.673344609485929e-06
Diff: (-7.988964805112988e-07+0j)
Percent Diff: (27.793038686569734+0j)


# NW no Shell

In [17]:
conf = Config('AbsorptionTestnoshell.yml')
sim = Simulator(conf)
sim = setup_sim(sim)
Zo = consts.physical_constants['characteristic impedance of vacuum'][0]

In [18]:
sim.get_fluxes()

KeyboardInterrupt: 

In [ ]:
abs_dict_fluxmethod = compute_fluxes(sim)

In [ ]:
sim.get_field()

In [ ]:
sim_proc = Simulation(simulator=sim)
freq = sim_proc.conf[('Simulation', 'params', 'frequency','value')]
try:
    Esq = sim_proc.data['normEsquared']
except KeyError:
    Esq = sim_proc.normEsquared()
abs_dict_intmethod = {}
for layer_name, layer_obj in sim_proc.layers.items():
    print("Layer: {}".format(layer_name))
    base_unit = sim_proc.conf[('Simulation', 'base_unit')]
    n_mat, k_mat = layer_obj.get_nk_matrix(freq)
    # n and k could be functions of space, so we need to multiply the
    # fields by n and k before integrating
    res = integrate(Esq*n_mat*k_mat,layer_obj, sim_proc)
    p_abs_imag = 2*np.pi*freq*consts.epsilon_0*res*base_unit
    abs_dict_intmethod[layer_name] = p_abs_imag
    disp.display_latex("$P_{abs} = \\frac{\omega}{2} Im(\epsilon) \int |E|^2 dV"+" = {}$".format(p_abs_imag), raw=True)

In [ ]:
print(abs_dict_fluxmethod)
print(abs_dict_intmethod)

In [ ]:
for key in abs_dict_fluxmethod.keys():
    fm = abs_dict_fluxmethod[key]
    im = abs_dict_intmethod[key]
    diff = fm - im
    try:
        pdiff = 100*abs(diff)/fm
    except ZeroDivisionError:
        pdiff = None
        pass
    print('-'*25)
    print("Layer: {}".format(key))
    print("Flux Method: {}".format(fm))
    print("Integral Method: {}".format(im))
    print("Diff: {}".format(diff))
    print("Percent Diff: {}".format(pdiff))

In [ ]:
print(sim.data['fluxes'])

In [ ]:
print(sim.flux_dict)

In [ ]:
simulation = Simulation(simulator=sim)
print(simulation.data['fluxes'])
print(simulation.layers)

In [ ]:
alist = [1., 2., 3.]
print('{}, {}, {}, {}'.format('test', *alist))